In [19]:
import pandas as pd
import json
from justwatch import JustWatch
import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import get_from_table
from dotenv import load_dotenv


In [20]:
load_dotenv()

True

In [22]:
film_id = 'f_0jxmG'

In [23]:
with open('../data_prep/my_streaming_services.json', 'r') as schema:
    my_streaming_services = json.load(schema)
my_streaming_services_abbr = [x for x in set([x['provider_abbreviation'] for x in my_streaming_services]) if len(x) > 0]
abbr_to_full_dict = {x['provider_abbreviation']:x['streaming_service'] for x in my_streaming_services if len(x['provider_abbreviation']) > 0}
abbr_to_full_dict['rent'] = 'Rental'
just_watch = JustWatch(country='GB')
film_url_title = get_from_table('FILM_URL_TITLE', film_id, 'FILM_URL_TITLE')


In [24]:
if film_url_title == '':
    film_url_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
film_release_year = get_from_table('FILM_YEAR', film_id, 'FILM_YEAR')
# import ipdb; ipdb.set_trace()


In [28]:
results = just_watch.search_for_item(query=film_url_title, release_year_from=film_release_year-1, release_year_until=film_release_year+1, page_size=1)
first_result = results['items'][0]
first_result

{'jw_entity_id': 'tm939539',
 'id': 939539,
 'title': 'M3GAN',
 'full_path': '/uk/movie/m3gan',
 'full_paths': {'MOVIE_DETAIL_OVERVIEW': '/uk/movie/m3gan'},
 'poster': '/poster/302253576/{profile}',
 'poster_blur_hash': 'TqINjc-=XT~Wx^xu-;bwayxuWCM|',
 'original_release_year': 2022,
 'object_type': 'movie',
 'offers': [{'jw_entity_id': 'tm939539',
   'monetization_type': 'buy',
   'provider_id': 2,
   'package_short_name': 'itu',
   'retail_price': 13.99,
   'currency': 'GBP',
   'urls': {'standard_web': 'https://tv.apple.com/gb/movie/m3gan/umc.cmc.5rwomfz98u19czniaz4pdm5pc?playableId=tvs.sbd.9001%3A1668065018',
    'deeplink_android_tv': 'intent://tv.apple.com/gb/movie/m3gan/umc.cmc.5rwomfz98u19czniaz4pdm5pc?playableId=tvs.sbd.9001%3A1668065018#Intent;launchFlags=0x00800000;scheme=https;end',
    'deeplink_fire_tv': 'intent://tv.apple.com/gb/movie/m3gan/umc.cmc.5rwomfz98u19czniaz4pdm5pc?playableId=tvs.sbd.9001%3A1668065018#Intent;launchFlags=0x00800000;scheme=https;end',
    'deeplink

In [37]:
provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
print(provider_abbreviations)
valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
valid_abbr.append('rent')
print(valid_abbr)


[]
['rent']


In [41]:
[x['retail_price'] for x in first_result.get('offers', []) if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd' and x.get('retail_price', False)]

[4.99, 5.49, 4.99, 4.99, 4.49, 5.49]

In [ ]:

if len(results['items']) > 0:
    first_result = results['items'][0]
    if first_result.get('title') == get_from_table('FILM_TITLE', film_id, 'FILM_TITLE'):
        provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
        valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
        valid_abbr.append('rent')
        rental_prices = [x['retail_price'] for x in first_result.get('offers', []) if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd']
        if len(rental_prices) > 0:
            min_rental_price = min(rental_prices)
        else:
            min_rental_price = None
        valid_full = [abbr_to_full_dict.get(x) for x in valid_abbr]
        film_streaming_services_df = pd.DataFrame(index=range(len(valid_abbr)))
        film_streaming_services_df['FILM_ID'] = film_id
        film_streaming_services_df['STREAMING_SERVICE_ABBR'] = valid_abbr
        film_streaming_services_df['STREAMING_SERVICE_FULL'] = valid_full
        film_streaming_services_df['PRICE'] = np.where(film_streaming_services_df['STREAMING_SERVICE_ABBR']=='rent', min_rental_price, 0)
        film_streaming_services_df['CREATED_AT'] = datetime.now()

In [35]:
first_result = results['items'][0]
first_result

{'jw_entity_id': 'tm3293',
 'id': 3293,
 'title': 'The Passion of Joan of Arc',
 'full_path': '/uk/movie/the-passion-of-joan-of-arc',
 'full_paths': {'MOVIE_DETAIL_OVERVIEW': '/uk/movie/the-passion-of-joan-of-arc'},
 'poster': '/poster/35904680/{profile}',
 'poster_blur_hash': 'TRIOhGt7_3~qM{WBD%WBD%-;t7xu',
 'original_release_year': 1928,
 'object_type': 'movie',
 'offers': [{'jw_entity_id': 'tm3293',
   'monetization_type': 'flatrate',
   'provider_id': 224,
   'package_short_name': 'bfi',
   'currency': 'GBP',
   'urls': {'standard_web': 'https://player.bfi.org.uk/subscription/film/watch-the-passion-of-joan-of-arc-1928-online',
    'deeplink_tizenos': '{"id":"Rf144dmTlp.dist","action_data":"0cb52d1a-698b-5a1d-b826-f16390a89c7e"}'},
   'presentation_type': 'sd',
   'country': 'GB'},
  {'jw_entity_id': 'tm3293',
   'monetization_type': 'flatrate',
   'provider_id': 287,
   'package_short_name': 'abf',
   'currency': 'GBP',
   'urls': {'standard_web': 'https://watch.amazon.co.uk/detail

In [36]:
first_result.get('offers', [])

[{'jw_entity_id': 'tm3293',
  'monetization_type': 'flatrate',
  'provider_id': 224,
  'package_short_name': 'bfi',
  'currency': 'GBP',
  'urls': {'standard_web': 'https://player.bfi.org.uk/subscription/film/watch-the-passion-of-joan-of-arc-1928-online',
   'deeplink_tizenos': '{"id":"Rf144dmTlp.dist","action_data":"0cb52d1a-698b-5a1d-b826-f16390a89c7e"}'},
  'presentation_type': 'sd',
  'country': 'GB'},
 {'jw_entity_id': 'tm3293',
  'monetization_type': 'flatrate',
  'provider_id': 287,
  'package_short_name': 'abf',
  'currency': 'GBP',
  'urls': {'standard_web': 'https://watch.amazon.co.uk/detail?gti=amzn1.dv.gti.14b064e1-ea62-606e-cb50-d1cbf9a4d0e5',
   'deeplink_web': 'https://watch.amazon.co.uk/watch?gti=amzn1.dv.gti.14b064e1-ea62-606e-cb50-d1cbf9a4d0e5',
   'deeplink_android_tv': 'intent://watch.amazon.co.uk/watch?asin=B078TZ5KRX&time=0&territory=GB#Intent;package=com.amazon.amazonvideo.livingroom;scheme=https;end',
   'deeplink_fire_tv': 'intent://com.amazon.tv.launcher/detai

In [37]:
provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
valid_full = [abbr_to_full_dict.get(x) for x in valid_abbr]
film_streaming_services_df = pd.DataFrame(index=range(len(valid_abbr)))
film_streaming_services_df['FILM_ID'] = film_id
film_streaming_services_df['STREAMING_SERVICE_ABBR'] = valid_abbr
film_streaming_services_df['STREAMING_SERVICE_FULL'] = valid_full
min_rental_price = min([x['retail_price'] for x in first_result['offers'] if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd'])
min_rental_price

ValueError: min() arg is an empty sequence

In [20]:
min([x['retail_price'] for x in first_result['offers'] if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd'])

ValueError: min() arg is an empty sequence

In [17]:

# delete_records('FILM_STREAMING_SERVICES', film_id)
if len(results['items']) > 0:
    first_result = results['items'][0]
    if first_result.get('title') == get_from_table('FILM_TITLE', film_id, 'FILM_TITLE'):
        provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
        valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
        if len(valid_abbr) > 0:
            valid_full = [abbr_to_full_dict.get(x) for x in valid_abbr]
            film_streaming_services_df = pd.DataFrame(index=range(len(valid_abbr)))
            film_streaming_services_df['FILM_ID'] = film_id
            film_streaming_services_df['STREAMING_SERVICE_ABBR'] = valid_abbr
            film_streaming_services_df['STREAMING_SERVICE_FULL'] = valid_full
            # film_streaming_services_df['CREATED_AT'] = datetime.now()
            # df_to_table(film_streaming_services_df, 'FILM_STREAMING_SERVICES', replace_append='append', verbose=False)

In [39]:
min_rental_price = min([x['retail_price'] for x in first_result['offers'] if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd'])

In [23]:
[x for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]

[{'jw_entity_id': 'tm35196',
  'monetization_type': 'flatrate',
  'provider_id': 194,
  'package_short_name': 'ast',
  'currency': 'GBP',
  'urls': {'standard_web': 'https://watch.amazon.co.uk/detail?gti=amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d',
   'deeplink_web': 'https://watch.amazon.co.uk/watch?gti=amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d',
   'deeplink_android_tv': 'intent://watch.amazon.co.uk/watch?asin=B00L2OBRT6&time=0&territory=GB#Intent;package=com.amazon.amazonvideo.livingroom;scheme=https;end',
   'deeplink_fire_tv': 'intent://com.amazon.tv.launcher/detail?provider=aiv&providerId=B00L2OBRT6#Intent;package=com.amazon.tv.launcher;scheme=amzn;end',
   'deeplink_tvos': 'aiv://aiv/watch?gti=amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d&territory=GB&time=auto',
   'deeplink_tizenos': '{"id":"org.tizen.primevideo","action_data":"amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d"}',
   'deeplink_webos': '{ "id": "amazon", "params": { "contentTarget": "amzn1.dv